In [2]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import keras
from keras import backend as K

In [3]:
BACKBONE='resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [4]:
def jacard_coef(y_true, y_pred):
    # y_true=y_true.astype(np.uint8)
    # y_prede=y_pred.astype(np.uint8)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0))


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

In [5]:
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.5, 0.5])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [6]:
SIZE_X = 256
SIZE_Y = 256

In [9]:
train_images = []
train_labels=[]
for directory_path in (glob.glob("SBU-shadow/SBUTrain4KRecoveredSmall/ShadowImages")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
#         print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

NameError: name 'label' is not defined

In [10]:
train_masks = [] 
for directory_path in sorted(glob.glob("SBU-shadow/SBUTrain4KRecoveredSmall/ShadowMasks/")):
    for mask_path in sorted(glob.glob(os.path.join(directory_path, "*.png"))):
        print(mask_path)
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        train_labels.append(label)
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

SBU-shadow/SBUTrain4KRecoveredSmall/ShadowMasks/lssd1.png


NameError: name 'label' is not defined

In [7]:
X=np.load('imgs.npy')
Y=np.load('masks.npy')
Y = np.expand_dims(Y, axis=3)
Y=(Y>0.5).astype(np.float32)

In [5]:
# X=train_images
# Y=train_masks
# Y = np.expand_dims(Y, axis=3)

NameError: name 'train_images' is not defined

In [8]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

In [10]:
model = sm.Unet(BACKBONE,(256,256,3),classes=1,activation='sigmoid', encoder_weights='imagenet')
model.compile(optimizer= 'adam', loss=[jacard_coef_loss],
              metrics = [jacard_coef])
print(model.summary())

2022-08-17 16:18:28.862198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 16:18:29.530361: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pyrosama/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-08-17 16:18:29.530380: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-08-17 16:18:29.531473: I tensorflow/core/platform/cpu_feature_guard.cc:19

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 256, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 128, 128, 64  9408        ['zero_padding2d[0][0]']   

In [11]:
history=model.fit(x_train, 
          y_train,
          batch_size=16, 
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val))

Epoch 1/10
 40/205 [====>.........................] - ETA: 5:28 - loss: -0.2842 - jacard_coef: 0.2842

KeyboardInterrupt: 